In [2]:
import os
import csv

path = 'data/' 

samples = []
with open(path+'/driving_log.csv') as file:
    reader = csv.reader(file)
    for line in reader: 
        samples.append(line)

from sklearn.model_selection import train_test_split
train_samples, test_samples = train_test_split(samples, test_size=.2)
test_samples, valid_samples = train_test_split(samples, test_size=.25)

import numpy as np
import sklearn
import matplotlib.image as mpimg

def generator(samples, batch_size=32):
    while True: 
        sklearn.utils.shuffle(samples)
        for start in range(0, len(samples), batch_size//2): 
            batch_samples = samples[start:start+batch_size]

            images = []
            angles = []

            for batch_sample in batch_samples:
                name = path+'/IMG/'+batch_sample[0].split('\\')[-1]
                center_image = mpimg.imread(name)
                center_angle=float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)
                
                # flip image 
                center_image_flip = np.fliplr(center_image)
                center_angle_flip = -center_angle
                images.append(center_image_flip)
                angles.append(center_angle_flip)

            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

In [4]:
import keras 
import tensorflow as tf

input_shape = (160,320,3)

car_input = keras.layers.Input(shape=input_shape)

cropped_input = keras.layers.Cropping2D(cropping=((50, 25), (0, 0)))(car_input)

crop_input_shape = (85, 300, 3) # new input shape (85, 300, 3)

normal_input = keras.layers.Lambda(lambda x: (x/255.0)-.5)(cropped_input)

resize_factor = 139/85
resize_shape = np.int32(np.multiply(crop_input_shape,resize_factor))[:2]
print(resize_shape)

resized_input = keras.layers.Lambda(lambda image: tf.image.resize_images( 
    image, resize_shape))(normal_input)

[139 490]


In [8]:
# Loads in InceptionV3
from keras.applications.inception_v3 import InceptionV3

inception = InceptionV3(weights=None, include_top=False,
                        input_shape=(*resize_shape,3))

In [12]:
inp = inception(resized_input)

pool = keras.layers.GlobalAveragePooling2D()(inp)

fc = keras.layers.Dense(512, activation=tf.nn.relu)(pool)

prediction = keras.layers.Dense(1)(fc)

In [13]:
from keras.models import Model

# reates the model
model = Model(inputs=car_input, outputs=prediction)

# Compile the model
model.compile(optimizer='Adam', loss='mse', metrics=['accuracy'])

# Check the summary of model to confirm architecture
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_2 (Cropping2D)    (None, 85, 320, 3)        0         
_________________________________________________________________
lambda_3 (Lambda)            (None, 85, 320, 3)        0         
_________________________________________________________________
lambda_4 (Lambda)            (None, 139, 490, 3)       0         
_________________________________________________________________
inception_v3 (Model)         (None, 3, 13, 2048)       21802784  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               1049088   
__________

In [ ]:
batch_size = 128
from math import ceil

train_generator = generator(train_samples, batch_size=batch_size)
valid_generator = generator(valid_samples, batch_size=batch_size)

model.fit_generator(train_generator, 
            steps_per_epoch=ceil(len(train_samples)/batch_size), 
            validation_data=valid_generator, 
            validation_steps=ceil(len(valid_samples)/batch_size), 
            epochs=5, verbose=1)

model.save('model.h5')